In [1]:
# Import Dependencies
import pandas as pd 
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# import tensorflow as tf

In [2]:
# Read CSV file from Resources folder into Pandas DataFrame
file = Path('./Resources_sdt/survey.csv')
survey_df = pd.read_csv(file)

# Review DataFrame
survey_df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


In [3]:
# Dropping 'comments' column due to limited value-added data and droppint 'Timestamp' because the entire dataset is for 2014
# Adding option to drop 'state' to look at the dataset globally and due to the NaN values -- can comment out and rerun
survey_df.drop(columns=['comments', 'Timestamp'], inplace=True)
survey_df.drop(columns=['state'], inplace=True)

In [4]:
# Get DataFrame info
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Age                        1259 non-null   int64 
 1   Gender                     1259 non-null   object
 2   Country                    1259 non-null   object
 3   self_employed              1241 non-null   object
 4   family_history             1259 non-null   object
 5   treatment                  1259 non-null   object
 6   work_interfere             995 non-null    object
 7   no_employees               1259 non-null   object
 8   remote_work                1259 non-null   object
 9   tech_company               1259 non-null   object
 10  benefits                   1259 non-null   object
 11  care_options               1259 non-null   object
 12  wellness_program           1259 non-null   object
 13  seek_help                  1259 non-null   object
 14  anonymit

In [5]:
# Determine number of unique values in each column
survey_df.nunique()

Age                          53
Gender                       49
Country                      48
self_employed                 2
family_history                2
treatment                     2
work_interfere                4
no_employees                  6
remote_work                   2
tech_company                  2
benefits                      3
care_options                  3
wellness_program              3
seek_help                     3
anonymity                     3
leave                         5
mental_health_consequence     3
phys_health_consequence       3
coworkers                     3
supervisor                    3
mental_health_interview       3
phys_health_interview         3
mental_vs_physical            3
obs_consequence               2
dtype: int64

## Check Age Ranges within the Dataset
Verify ages for authenticity, clean data as needed.

In [6]:
# Look at 'Age' value counts
age_ranges = survey_df['Age'].value_counts()
age_ranges

Age
 29             85
 32             82
 26             75
 27             71
 33             70
 28             68
 31             67
 34             65
 30             63
 25             61
 35             55
 23             51
 24             46
 37             43
 38             39
 36             37
 40             33
 39             33
 43             28
 22             21
 41             21
 42             20
 21             16
 45             12
 46             12
 44             11
 19              9
 18              7
 48              6
 50              6
 20              6
 51              5
 49              4
 56              4
 57              3
 54              3
 55              3
 47              2
 60              2
 99999999999     1
 5               1
-1               1
 11              1
 8               1
 61              1
 53              1
-29              1
-1726            1
 65              1
 62              1
 58              1
 329             1
 72     

In [7]:
# Several answers for 'Age' were given with incorrect or bogus values. This is an attempt to retain the data and clean up the column.

# Convert the 'Age' column to numeric
survey_df['Age'] = pd.to_numeric(survey_df['Age'], errors='coerce')

# Then replace any ages less than 0 OR greater than 99 with 'Other'
survey_df.loc[(survey_df['Age'] < 0) | (survey_df['Age'] > 99), 'Age'] = 'Other'

# Check the updated value counts
age_update = survey_df['Age'].value_counts()
age_update

Age
29       85
32       82
26       75
27       71
33       70
28       68
31       67
34       65
30       63
25       61
35       55
23       51
24       46
37       43
38       39
36       37
40       33
39       33
43       28
22       21
41       21
42       20
21       16
45       12
46       12
44       11
19        9
18        7
50        6
48        6
20        6
Other     5
51        5
56        4
49        4
54        3
55        3
57        3
47        2
60        2
62        1
58        1
65        1
5         1
53        1
61        1
8         1
11        1
72        1
Name: count, dtype: int64

In [8]:
# check 'Age' column count
age_column = survey_df['Age'].count()
age_column

1259

## Gender Column Cleanup
There were 49 unique genders listed. Some of these were because of input errors. Some are because of how the person represented themselves. Need to determine which they are to correct them.

In [9]:
# Look at 'Gender' value counts
gender_count = survey_df['Gender'].value_counts()
gender_count

Gender
Male                                              615
male                                              206
Female                                            121
M                                                 116
female                                             62
F                                                  38
m                                                  34
f                                                  15
Make                                                4
Male                                                3
Woman                                               3
Cis Male                                            2
Man                                                 2
Female (trans)                                      2
Female                                              2
Trans woman                                         1
msle                                                1
male leaning androgynous                            1
Neuter               

In [10]:
# There may have been misspellings in 'Gender' names and in some cases bogus answers. This is an attempt to cleanup the 'Gender' to form usable data.

# Adding Category for 'Male'
survey_df['Gender'].replace(['Male', 'male', 'M', 'm', 'Male ', 'Cis Male', 'cis male', 'Man', 
                            'Make', 'Mail', 'Cis Man', 'Guy (-ish) ^_^', 'Male-ish', 'maile', 
                            'Mal', 'Male (CIS)', 'msle', 'Malr'], 'Male', inplace=True)

# Adding Category for 'Female'
survey_df['Gender'].replace(['Female', 'female', 'F', 'f', 'Woman', 'Female ', 'Female (cis)', 
                            'cis-female/femme', 'femail', 'Cis Female', 'Femake', 'woman'], 'Female', inplace=True)

# Adding Category for 'Other' -- Note: Other will include non-specific gender and trans
survey_df['Gender'].replace(['Nah', 'All', 'fluid', 'Genderqueer', 'ostensibly male, unsure what that really means', 
                            'non-binary', 'Androgyne', 'Agender', 'Enby', 'p', 'Neuter', 'queer', 'Trans woman', 
                            'Female (trans)', 'male leaning androgynous', 'A little about you', 'Trans-female', 
                            'something kinda male?', 'queer/she/they'], 'Other', inplace=True)

# Check the updated value counts
gender_update = survey_df['Gender'].value_counts()
gender_update

Gender
Male      992
Female    247
Other      20
Name: count, dtype: int64

In [11]:
# check 'Gender' column count
gender_column = survey_df['Gender'].count()
gender_column

1259

## Replace NULL Values in the Dataset

In [12]:
# Replacing the NaN in the 'self_employed' and 'work_interfere' to a 'no' answer, assuming a non-answer means "No".
survey_df = survey_df.fillna({'self_employed': 'No',
                            'work_interfere': 'Never'})

survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Age                        1259 non-null   object
 1   Gender                     1259 non-null   object
 2   Country                    1259 non-null   object
 3   self_employed              1259 non-null   object
 4   family_history             1259 non-null   object
 5   treatment                  1259 non-null   object
 6   work_interfere             1259 non-null   object
 7   no_employees               1259 non-null   object
 8   remote_work                1259 non-null   object
 9   tech_company               1259 non-null   object
 10  benefits                   1259 non-null   object
 11  care_options               1259 non-null   object
 12  wellness_program           1259 non-null   object
 13  seek_help                  1259 non-null   object
 14  anonymit